In [ ]:
! pip install transformers pytorch_lightning hf-hub-lightning datasets sentencepiece tokenizers
! apt install git-lfs
! git config --global credential.helper store
! huggingface-cli login

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 16.0 MB/s 
     |████████████████████████████████| 708 kB 81.6 MB/s 
     |████████████████████████████████| 431 kB 58.6 MB/s 
     |████████████████████████████████| 1.3 MB 84.4 MB/s 
     |████████████████████████████████| 7.0 MB 82.6 MB/s 
     |████████████████████████████████| 6.6 MB 64.1 MB/s 
     |████████████████████████████████| 163 kB 96.0 MB/s 
     |████████████████████████████████| 5.9 MB 35.4 MB/s 
     |████████████████████████████████| 419 kB 98.5 MB/s 
     |████████████████████████████████| 212 kB 85.7 MB/s 
     |████████████████████████████████| 115 kB 99.9 MB/s 
     |████████████████████████████████| 127 kB 93.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: tensorboard
    Found 

In [ ]:
class Config:
  DATASET_ID = "mlqa"
  SUBSET_ID = "mlqa.en.hi"
  TRAIN_DATA_PATH = "/content/artifacts/data/train.csv"
  TEST_DATA_PATH = "/content/artifacts/data/test.csv"
  VALID_DATA_PATH = "/content/artifacts/data/valid.csv"
  MODEL_CKPT = "google/mt5-base"
  QUESTION_MAX_LEN = 512
  ANSWER_MAX_LEN = 64
  BATCH_SIZE = 4
  MODEL_OUT = "Vasanth/mlqa-model"
  NUM_EPOCHS = 5

In [ ]:
config = Config()

In [ ]:
# import config
from datasets import load_dataset
import pandas as pd

class DataGen:
    
    def __init__(self):
        self.dataset_id = config.DATASET_ID
        self.subset_id = config.SUBSET_ID
    
    def load_data(self):
        train_test_data = load_dataset(self.dataset_id, self.subset_id, split="test")
        valid_data = load_dataset(self.dataset_id, self.subset_id, split="validation")
        return train_test_data[:4400], train_test_data[4400:], valid_data
        
    def data_df(self):
        train_data, test_data, valid_data = self.load_data()
        self.df_train = pd.DataFrame(train_data)
        self.df_test = pd.DataFrame(test_data)
        self.df_valid = pd.DataFrame(valid_data)
    
    def final_data_prep(self):
        self.df_train = self.df_train.drop("id", axis=1)
        self.df_train["answers"] = self.df_train["answers"].apply(lambda x: x["text"][0])
        self.df_test = self.df_test.drop("id", axis=1)
        self.df_test["answers"] = self.df_test["answers"].apply(lambda x: x["text"][0])
        self.df_valid = self.df_valid.drop("id", axis=1)
        self.df_valid["answers"] = self.df_valid["answers"].apply(lambda x: x["text"][0])
    
    def create_data_csv(self):
        self.data_df()
        self.final_data_prep()
        self.df_train.to_csv(config.TRAIN_DATA_PATH, index=False)
        self.df_test.to_csv(config.TEST_DATA_PATH, index=False)
        self.df_valid.to_csv(config.VALID_DATA_PATH, index=False)

In [ ]:
# import config
import pandas as pd
from torch.utils.data import Dataset
from transformers import MT5Tokenizer
import pytorch_lightning as pl


class MLQADataset(Dataset):

    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: MT5Tokenizer,
        question_max_token_len = config.QUESTION_MAX_LEN,
        answer_max_token_len = config.ANSWER_MAX_LEN
                    ):
        self.tokenizer = tokenizer
        self.data = data
        self.max_qns_len = question_max_token_len
        self.max_ans_len = answer_max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        data_idx = self.data.iloc[index]

        question_encoding = self.tokenizer(
            data_idx["question"],
            data_idx["context"],
            max_length=config.QUESTION_MAX_LEN,
            padding="max_length",
            truncation="only_second",
            add_special_tokens=True,
            return_attention_mask=True,
            return_tensors="pt"   
        )

        answer_encoding = self.tokenizer(
            data_idx["answers"],
            max_length=config.ANSWER_MAX_LEN,
            padding="max_length",
            truncation=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_tensors="pt"   
        )

        labels = answer_encoding["input_ids"]
        labels[labels==0] = -100

        return dict (
            context = data_idx["context"],
            question = data_idx["question"],
            answer = data_idx["answers"],
            input_ids = question_encoding["input_ids"].flatten(),
            attention_mask = question_encoding["attention_mask"].flatten(),
            labels = labels.flatten()
        )

In [ ]:
# import config
import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule
import pandas as pd
from transformers import MT5Tokenizer
# from dataset import MLQADataset
from torch.utils.data import DataLoader
import torch
import gc

class MultiQADataLoader(LightningDataModule):

    def __init__(
        self,
        ):
        super().__init__()
        self.batch_size = config.BATCH_SIZE
        self.train_df = pd.read_csv(config.TRAIN_DATA_PATH)
        self.test_df = pd.read_csv(config.TEST_DATA_PATH)
        self.valid_df = pd.read_csv(config.VALID_DATA_PATH)
        self.tokenizer = MT5Tokenizer.from_pretrained(config.MODEL_CKPT)
        self.src_max_token_len = config.QUESTION_MAX_LEN
        self.tgt_max_token_len = config.ANSWER_MAX_LEN

    def setup(self, stage):
        self.train_dataset = MLQADataset(
            self.train_df,
            self.tokenizer
        )

        self.test_dataset = MLQADataset(
            self.test_df,
            self.tokenizer
        )

        self.val_dataset = MLQADataset(
            self.valid_df,
            self.tokenizer
        )

    def train_dataloader(self):
        torch.cuda.empty_cache()
        gc.collect()
        return DataLoader(
            self.train_dataset,
            batch_size=1,
            shuffle=True,
            num_workers=4
        )

    def test_dataloader(self):
        torch.cuda.empty_cache()
        gc.collect()
        return DataLoader(
            self.test_dataset,
            batch_size=1,
            num_workers=4
        )
    
    def val_dataloader(self):
        torch.cuda.empty_cache()
        gc.collect()
        return DataLoader(
            self.val_dataset,
            batch_size=1,
            num_workers=4
        )

In [ ]:
# import config
import pytorch_lightning as pl
from pytorch_lightning import LightningModule
from transformers import MT5ForConditionalGeneration, AdamW
import torch
import gc

class Model(LightningModule):

    def __init__(self):
        super().__init__()
        torch.cuda.empty_cache()
        gc.collect()
        self.model = MT5ForConditionalGeneration.from_pretrained(config.MODEL_CKPT, return_dict=True)
        torch.cuda.empty_cache()
        gc.collect()

    def forward(self, input_ids, attention_mask, labels=None):
        torch.cuda.empty_cache()
        gc.collect()
        output = self.model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            labels = labels
        )

        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        torch.cuda.empty_cache()
        gc.collect()
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=False)
        return loss

    def validation_step(self, batch, batch_idx):
        torch.cuda.empty_cache()
        gc.collect()
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=False)
        return loss

    def test_step(self, batch, batch_idx):
        torch.cuda.empty_cache()
        gc.collect()
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("test_loss", loss, prog_bar=True, logger=False)
        return loss
    
    def configure_optimizers(self):
        torch.cuda.empty_cache()
        gc.collect()
        return AdamW(self.parameters(), lr=0.0001)
        
        

In [ ]:
# import config
import pandas as pd
import pytorch_lightning as pl
from hf_hub_lightning import HuggingFaceHubCallback
import torch
import gc

if __name__ == "__main__":

    print("Start")
    pl.seed_everything(0)
    
    datagen = DataGen()
    datagen.create_data_csv()

    torch.cuda.empty_cache()
    gc.collect()

    print("Data CSV Created")

    multiqadataloader = MultiQADataLoader()
    multiqadataloader.setup(stage="fit")

    torch.cuda.empty_cache()
    gc.collect()


    print("Data Loader Set")

    model = Model()

    torch.cuda.empty_cache()
    gc.collect()

    trainer = pl.Trainer(
        callbacks=[HuggingFaceHubCallback(config.MODEL_OUT)],
        max_epochs=config.NUM_EPOCHS,
        gpus=1
    )
    
    torch.cuda.empty_cache()
    gc.collect()

    trainer.fit(model, multiqadataloader)

INFO:pytorch_lightning.utilities.seed:Global seed set to 0


Start


Data CSV Created


Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Vasanth/mlqa-model
   c46953d..16dc104  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Vasanth/mlqa-model
   c46953d..16dc104  main -> main



Validation: 0it [00:00, ?it/s]

Pulling changes ...


Upload file lit_model.ckpt:   0%|          | 1.00/6.51G [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Vasanth/mlqa-model
   16dc104..0df75ee  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Vasanth/mlqa-model
   16dc104..0df75ee  main -> main

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
# import config
from transformers import MT5Tokenizer
from huggingface_hub import hf_hub_download
# from model import Model
import pandas as pd

def generate_answer(question, model, tokenizer):
    question_encoding = tokenizer(
                question["question"],
                question["context"],
                max_length=config.QUESTION_MAX_LEN,
                padding="max_length",
                truncation="only_second",
                add_special_tokens=True,
                return_attention_mask=True,
                return_tensors="pt"   
            )
    
    gen_ids = model.model.generate(
        input_ids = question_encoding["input_ids"],
        attention_mask = question_encoding["attention_mask"],
        num_beams = 1,
        max_length = config.ANSWER_MAX_LEN,
        repetition_penalty = 2.5,
        length_penalty = 1.0,
        early_stopping = True,
        use_cache = True
    )

    preds = [
        tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for gen_id in gen_ids
    ]
    print("Context is:", question["context"])
    print("Question is :", question["question"])
    print("Original answer: ",question["answers"])
    return "Predicted answer: "+"".join(preds)

ckpt_path = hf_hub_download(config.MODEL_OUT, 'lit_model.ckpt')
model = Model.load_from_checkpoint(ckpt_path)
model.freeze()
tokenizer = MT5Tokenizer.from_pretrained(config.MODEL_CKPT)

test = pd.read_csv(config.TEST_DATA_PATH)
question = test.iloc[1]
print(generate_answer(question, model, tokenizer))

Context is: Mercury is tidally locked with the Sun in a 3:2 spin-orbit resonance, and rotates in a way that is unique in the Solar System. As seen relative to the fixed stars, it rotates on its axis exactly three times for every two revolutions it makes around the Sun.  As seen from the Sun, in a frame of reference that rotates with the orbital motion, it appears to rotate only once every two Mercurian years. An observer on Mercury would therefore see only one day every two Mercurian years.
Question is : बुध पर कोई व्यक्ति कितनी बार एक दिन का अनुभव करेगा?
Original answer:  every two Mercurian years
Predicted answer: every two Mercurian years
